In [2]:
# !pip install selenium
# !pip install webdriver_manager
# !pip install bs4
# !pip install pandas

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/3f/fd/c2e7bb547b5b96c7bd536b4a80c4564b7ce5cd38d10095fbba8648996ab9/selenium-4.18.1-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/14/fb/9299cf74953f473a15accfdbe2c15218e766bae8c796f2567c83bae03e98/trio-0.24.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for outcome from https://files.pythonhosted.org/packages/55/8b/5ab7257531a5d830fc8000c476e63c935488d74609b50f9384a643ec0a62/outcome-1.3.0.post0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for wsproto>=0.14 from https://files.pythonhosted.org/packages/78/58/e860788190eba3bcce367f74d29c4675466ce8dddfba85f7827588416f01/wspro

In [2]:
# conda install selenium

In [3]:
import time
import pandas as pd
import random

import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver  # 동적크롤링
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [2]:
# 데이터 불러오기
# df = pd.read_csv('./Data/전국_일반음식점.csv')
# df = pd.read_csv('./Data/강남_일반음식점.csv')
df = pd.read_csv('./Data/제주_일반음식점.csv')

In [3]:
df.head()

,사업장명,업태구분명,소재지전화,도로명전체주소
0,제주고세호해장국사계점,한식,NaN,"제주특별자치도 서귀포시 안덕면 사계남로153번길 2, 1층"
1,어우름성산흑돼지,식육(숯불구이),NaN,"제주특별자치도 서귀포시 성산읍 일출로 292, 1층"
2,떡볶이삼춘,분식,NaN,제주특별자치도 서귀포시 하논로67번길 43 (서홍동)
3,카메오,기타,NaN,"제주특별자치도 제주시 서광로28길 21-1, 1층 (이도이동)"
4,돈까스튀기는서대리,한식,NaN,"제주특별자치도 제주시 선덕로8길 26, 가동 1층 103호 (연동, 비산쉐르빌)"


In [7]:
# import selenium
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys

# driver = webdriver.Chrome() (윈도우용)

In [4]:
# 크롬 드라이버 실행 (맥용)
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=chrome_options)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
count = 1


In [5]:
# 리뷰를 추출해주는 함수
def extract_review():
    # 리뷰 추출
    rev = []  # 추출한 리뷰 저장
    for i in range(1, 11): # 더보기 누르지 않은 상태로 최대 10개
        try:  # 사진 있는 후기는 div 3번째에 텍스트 위치
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
            # print('리뷰가 저장됨')
            time.sleep(random.uniform(1,2))
            comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰
            # print('리뷰가 저장됨')
            rev.append(comment)
        except: # 사진 없는 후기는 div 2번째에 텍스트가 위치
            try: # 리뷰에 글이 없는경우에 XPATH가 달라짐으로 try - except 추가
                driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
                time.sleep(random.uniform(1,2))
                comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                # print('리뷰가 저장됨')
                rev.append(comment)
            except:
                # print('이상한 리뷰임')
                rev.append(pd.NA)

    # 더 많은 리뷰가 있는지 확인하고, 있을 경우에는 추가 리뷰를 가져오기 위해 "더보기" 버튼 클릭
    try:
        driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a').send_keys(Keys.ENTER) # 더보기 버튼 클릭으로 최대 20개까지 가져오기
        time.sleep(random.uniform(1,2))
        for i in range(11, 21):
            try:  # 사진 있는 후기는 div 3번째에 텍스트 위치
                driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
                # print('리뷰가 저장됨')
                time.sleep(random.uniform(1,2))
                comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰
                # print('리뷰가 저장됨')
                rev.append(comment)
            except: # 사진 없는 후기는 div 2번째에 텍스트가 위치
                try: # 리뷰에 글이 없는경우에 XPATH가 달라짐으로 try - except 추가
                    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
                    time.sleep(random.uniform(1,2))
                    comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                    # print('리뷰가 저장됨')
                    rev.append(comment)
                except:
                    # print('이상한 리뷰임')
                    rev.append(pd.NA)

    except:
        print('더 이상 리뷰가 없음')
    return rev

In [6]:
# 숙소 리뷰를 저장할 리스트
crawlingData = []

In [7]:
for i in df['사업장명']:
    url = "https://map.naver.com/v5/search/" + str(i)
    # # 검색창에 입력하지 않고 직접 해당 음식점의 주소로 이동 (정확도를 높이기 위해 강남점처럼 ~점으로 끝나는게 없으면 강남구 추가)
    # if not i.endswith('점'):
    #     url = "https://map.naver.com/v5/search/" + str(i) + " 강남구"
    # else:
    #     url = "https://map.naver.com/v5/search/" + str(i)
    # url = "https://map.naver.com/v5/search/국풍설렁탕(수암본점)"
    # url = "https://map.naver.com/v5/search/정식당 강남구"
    driver.get(url)
    time.sleep(random.uniform(2,3))


    try:
        driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]')) # iframe 이동
        time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # 리뷰 탭으로 이동
        lists = soup.select('#app-root > div > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-child(3)')
        # 별점/방문자리뷰/블로그리뷰 순일때 방문자리뷰는 두번째에 위치=span[2]
        if len(lists) > 2:
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a').send_keys(Keys.ENTER) # 방문자 리뷰
            time.sleep(random.uniform(1,2))

        # 방문자리뷰/블로그리뷰 순일때 방문자리뷰는 첫번째에 위치=span[1]
        else:
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/a').send_keys(Keys.ENTER) # 방문자 리뷰
            time.sleep(random.uniform(1,2))

        driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
        time.sleep(random.uniform(1,2))
        review = extract_review() # 리뷰 추출 함수 호출
        print(f'현재까지 크롤링된 데이터 갯수: {count}')
        # print(review)
        crawlingData.append(review)
        count += 1
    except:
        crawlingData.append('')
        print(f'현재까지 크롤링된 데이터 갯수: {count}')
        count += 1


    result_df = pd.DataFrame({'리뷰': crawlingData})

            


현재까지 크롤링된 데이터 갯수: 1
현재까지 크롤링된 데이터 갯수: 2
현재까지 크롤링된 데이터 갯수: 3
현재까지 크롤링된 데이터 갯수: 4
현재까지 크롤링된 데이터 갯수: 5


In [10]:
sum(result_df['리뷰'] == "")

310

---
# 크롤링이 너무 오래걸리는 관계로 테스트로 239개의 데이터로 진행해보기

In [11]:
result_df.to_csv('./Data/제주_리뷰Columns.csv', index=False)

In [ ]:
# 기존의 데이터 239개만 추출
# test_df = df[0:239]

In [ ]:
# 기존의 데이터에 컬럼으로 크롤링해온 리뷰를 추가
# test_df['리뷰'] = result_df
# df['리뷰'] = result_df

In [ ]:
# 크롤링해온 데이터 중에서 리뷰가 없는 행들은 제거
# jeju_restaurant_df = test_df[test_df['리뷰'] != ""]
# jeju_restaurant_df = df[df['리뷰'] != ""]

> 9시간 크롤링 결과가 82개의 데이터만 남게됨..

In [ ]:
# 결측치 확인 결과 전화번호만 결측치가 있어서 상관 없음
# jeju_restaurant_df.isna().sum()

사업장명        0
소재지전화      73
도로명전체주소     0
좌표정보(x)     0
좌표정보(y)     0
리뷰          0
dtype: int64

In [ ]:
# csv파일로 변환
# gangnam_restaurant_df.to_csv('./Data/강남_82개_일반음식점.csv', index=False)
# jeju_restaurant_df.to_csv('./Data/제주_일반음식점_리뷰포함.csv', index=False)